# The Caring Home / Helpful Home Continued

A model that can look for non normal situations in a elderly person home, from smart home IoT sensors.

## Use AI to scan log file

We will introduce a concept of Structured Responses!!

In [194]:
# imports

import os
import json
import re
from typing import Optional, List
from openai import OpenAI
from agents.situations import LoadedSituation, SituationSelection
from agents.agent import Agent
from datetime import datetime, timedelta
from dotenv import load_dotenv


In [195]:
# environment

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['OPENROUTER_API_KEY'] = os.getenv('OPENROUTER_API_KEY', 'your-key-if-not-using-env')
os.environ['HYPERBOLIC_API_KEY'] = os.getenv('HYPERBOLIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [196]:
openai = OpenAI()
claude = Anthropic()

In [197]:
TIMEZONE = datetime.now().astimezone().tzinfo

SYSTEM_PROMPT = f"The timezone here is {str(TIMEZONE)}. "
SYSTEM_PROMPT += """Your task is to analyze log entries from home sensors and identify the most important situation involving an elderly person's movements that may require monitoring. You should:  

1. Select the situations with the most unusual, non-typical, or potentially concerning movement patterns.  
2. Return either a "normal" or "anomalous" result, strictly when confident.  
3. Ensure the structured output includes:  
- "situation_description": A concise and clear summary of the situation in 4-5 sentences. Focus solely on describing events and their significance.  
- "result": Either "normal" or "anomalous".  
- "start_timestamp" and "end_timestamp": Exact timestamps from the logs marking the situation’s beginning and end.  
- "details": a list of log entries that describe the situation that occured between the identified start and end timestamps. Each log entry should include the original details such as {"timestamp": 1737900000, "room": "pillbox", "nodeId": 1, "onOff": true}

**Important rules:**  

- Do not mention specific times or dates in the situation description unless they are fully aligned with the given start and end timestamps.  
- If times are included, ensure they match the provided start and end timestamps after conversion to human-readable format.  
- Respond only when highly confident about the situation's result and timestamp accuracy.  
- When returning the details of the situation, ensure that the log entries are escaped appropriately to allow these events to be easily parsed into json.
- Only include 1 situation in the response.

The output format should strictly be:  

{
"situations": [
{
  "situation_description": "Summary of the most concerning situation based only on log events.",
  "result": "normal | anomalous",
  "start_timestamp": "exact log entry timestamp marking the start",
  "end_timestamp": "exact log entry timestamp marking the end",
  "details": "list of log entries that relate to this situation"
}
]
}
"""

USER_PROMPT_PREFIX = """Analyze the following 6-hour block of log entries. Identify the most unusual or concerning movement of the elderly person. Summarize the situation concisely and ensure all timestamps in the response are correct and consistent.
Do not mention timestamps in the situation description unless they align precisely with the provided start and end timestamps.
Here are the log entries:

"""

USER_PROMPT_SUFFIX = "\n\nStrictly respond in valid JSON format, and only JSON and include exactly 1 situation, no more."



In [198]:
filename = "data/daily_routine_data.json"

INVESTIGATON_PERIOD_IN_HOURS = 6

def parse():
    """
    Parse the input file and group events by a 6-hour window.
    """
    if not os.path.exists(filename):
        return []

    with open(filename, 'r') as file:
        # Load and sort events by timestamp
        try:
            events = [json.loads(line) for line in file]
        except json.JSONDecodeError:
            print("error")
            return []

        events.sort(key=lambda x: datetime.fromtimestamp(x["timestamp"]))
    
        grouped = []
        window_start = None
        buffer = []
    
        # Group events in 6-hour windows
        for event in events:
            event_time = datetime.fromtimestamp(event["timestamp"])
            
            # Start or reset grouping window
            if window_start is None or event_time > window_start + timedelta(hours=INVESTIGATON_PERIOD_IN_HOURS):
                if buffer:
                    grouped.append({"details": buffer.copy()})
                buffer.clear()
                window_start = event_time
    
            buffer.append(event)
    
        # Append any remaining events in the buffer
        if buffer:
            grouped.append({"details": buffer})

        return grouped

    return []


In [199]:
loaded = parse()
print('\n\n'.join([json.dumps(situation) for situation in loaded]))

{"details": [{"timestamp": 1737900000, "room": "pillbox", "nodeId": 1, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737900060, "room": "bedroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737900300, "room": "bathroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737900600, "room": "bathroom", "nodeId": 3, "endpointId": 1, "attribute": {"RelativeHumidityMeasurement": {"MeasuredValue": 6500}}}, {"timestamp": 1737900900, "room": "bathroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}}, {"timestamp": 1737901200, "room": "kitchen", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737901260, "room": "kitchen", "nodeId": 2, "endpointId": 1, "attribute": {"Event": {"Event": "fridge_opened"}}}, {"timestamp": 1737901800, "room": "kitchen", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": false}}}, {"tim

In [200]:
def make_user_prompt(loaded) -> str:
    """
    Create a user prompt for OpenAI based on the scraped deals provided
    """
    user_prompt = USER_PROMPT_PREFIX
    user_prompt += '\n\n'.join([json.dumps(situation) for situation in loaded])
    user_prompt += USER_PROMPT_SUFFIX
    return user_prompt

In [201]:
make_user_prompt(loaded)

'Analyze the following 6-hour block of log entries. Identify the most unusual or concerning movement of the elderly person. Summarize the situation concisely and ensure all timestamps in the response are correct and consistent.\nDo not mention timestamps in the situation description unless they align precisely with the provided start and end timestamps.\nHere are the log entries:\n\n{"details": [{"timestamp": 1737900000, "room": "pillbox", "nodeId": 1, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737900060, "room": "bedroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737900300, "room": "bathroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"OnOff": true}}}, {"timestamp": 1737900600, "room": "bathroom", "nodeId": 3, "endpointId": 1, "attribute": {"RelativeHumidityMeasurement": {"MeasuredValue": 6500}}}, {"timestamp": 1737900900, "room": "bathroom", "nodeId": 2, "endpointId": 1, "attribute": {"OnOff": {"


## Structure Response using Pydantic Classes
We need this to tell how the LLM should respond with Structured Responses!!

In [202]:
from pydantic import BaseModel


class Situation(BaseModel):
    """
    A class to Represent a Situation with a summary description
    """
    situation_description: str
    result: str
    start_timestamp: int
    end_timestamp: int
    details: List[str]

class SituationSelection(BaseModel):
    """
    A class to Represent a list of Situations
    """
    situations: List[Situation]

class Investigation(BaseModel):
    """
    A class to represent a possible situation: a Situation where we estimate
    it could be anomalous
    """
    situation: Situation
    estimate: str


In [203]:
result = openai.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": make_user_prompt(loaded)}
    ],
    seed=42,
    response_format=SituationSelection
)

result = result.choices[0].message.parsed
result.situations = [situation for situation in result.situations if situation.result is not None]

In [204]:
print(result)

situations=[Situation(situation_description='An elderly person demonstrated notable movement patterns spanning several hours, particularly an extended period of inactivity indicated by long durations of occupancy sensing being low in the hall and porch areas. The log shows a significant increase in movement across multiple rooms, particularly in the kitchen, bathroom, and living room, with multiple instances of the fridge being opened. This may suggest irregular behaviors, as typical patterns for an elderly person usually involve shorter activity spans without such extended movements or frequent refrigerator access. Continuous monitoring may be necessary during such periods to ensure overall safety.', result='anomalous', start_timestamp=1738592400, end_timestamp=1738599600, details=['{"timestamp": 1738592400, "room": "porch", "nodeId": 1, "endpointId": 1, "attribute": {"OccupancySensing": {"Occupancy": 1}}}', '{"timestamp": 1738592460, "room": "hall", "nodeId": 1, "endpointId": 1, "att